In [2]:
import math
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import TensorDataset

In [3]:
print(np.arange(20))
poly_features = np.power(3, np.arange(20))
print(poly_features)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[         1          3          9         27         81        243
        729       2187       6561      19683      59049     177147
     531441    1594323    4782969   14348907   43046721  129140163
  387420489 1162261467]


In [4]:
max_degree = 20  # 多项式的最大阶数
n_train, n_test = 100, 100  # 训练和测试数据集大小
true_w = np.zeros(max_degree)  # 分配大量的空间
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])

features = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(features)
poly_features = np.power(features, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # gamma(n)=(n-1)!
# labels的维度:(n_train+n_test,)
labels = np.dot(poly_features, true_w) 
print(labels)
print(len(labels))
labels += np.random.normal(scale=0.1, size=labels.shape)


[  5.98116748   5.53454648   4.49673763   2.29901218   3.01269733
   5.39262599   5.08779548   6.94132676   5.26135316   7.2963345
   5.219275     4.87145041   5.60646233   5.3154984    2.91210804
   4.84168084   5.25806667  -0.96649512   5.41085954   4.66282728
   4.76232116   5.51618068   6.19052242   5.31237867  -0.67424944
   1.21102062   3.91091262   1.59639953  -5.73950553   4.46380425
  -0.49230668   6.51607459   5.30437479   4.93873335   4.18037538
 -25.91750836 -24.29388588   5.02769374  -3.8792823    1.54103988
  -1.46802319   5.15195526   5.16613381   5.29445411   4.94624803
   4.33996959   7.5126955   -2.41888706   3.95565043   1.50077017
   4.51137846   4.3982749    5.1197532    4.58867247   5.35083343
   4.99525279   5.27607887   5.47406004   4.93953359   5.4797279
  -2.95612254   5.13877434  -5.25001925   5.25913667   5.24308424
  -8.06312627   4.46912989   5.18556833   5.32602981   5.29816849
  -6.59871569 -10.03242255   6.28120187   5.16873906   5.20152104
   5.5798665

In [5]:
# NumPy ndarray转换为tensor
numpy_arrays = [true_w, features, poly_features, labels]
tensors = [torch.from_numpy(arr) for arr in numpy_arrays]
true_w_tensor, features_tensor, poly_features_tensor, labels_tensor = tensors


features[:2], poly_features[:2, :], labels[:2]

(array([[1.43589105],
        [1.13274457]]),
 array([[1.00000000e+00, 1.43589105e+00, 1.03089155e+00, 4.93415981e-01,
         1.77122897e-01, 5.08658364e-02, 1.21729665e-02, 2.49700766e-03,
         4.48178867e-04, 7.15040024e-05, 1.02671957e-05, 1.34023403e-06,
         1.60369170e-07, 1.77132812e-08, 1.81673870e-09, 1.73909256e-10,
         1.56071714e-11, 1.31824692e-12, 1.05158831e-13, 7.94719071e-15],
        [1.00000000e+00, 1.13274457e+00, 6.41555130e-01, 2.42239363e-01,
         6.85988309e-02, 1.55409906e-02, 2.93399546e-03, 4.74781061e-04,
         6.72257086e-05, 8.46106182e-06, 9.58422183e-07, 9.86952294e-08,
         9.31637377e-09, 8.11774754e-10, 6.56809603e-11, 4.95998341e-12,
         3.51149642e-13, 2.33978147e-14, 1.47243042e-15, 8.77835561e-17]]),
 array([6.03297329, 5.5583457 ]))

In [6]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
def evaluate_loss(net, data_iter, loss):  #@save
    """评估给定数据集上模型的损失"""
    metric = Accumulator(2)  # 损失的总和,样本数量
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum())
    return metric[0]
def train_epoch_ch3(net, train_iter, loss,trainer):  #@save
    """训练模型一个迭代周期（定义见第3章）"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    critizen = nn.MSELoss(reduction='none')
    #updater=optim.SGD(net.parameters, 0.1, 20)
    #X,y=next(iter(train_iter))
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        

        l=critizen(y_hat, y)
        trainer.zero_grad()
        l.backward(torch.ones_like(y_hat))
        trainer.step()
    # else:
    #     # 使用定制的优化器和损失函数
    #     l.sum().backward()
    #     updater(X.shape[0])
    metric.add(float(l.sum()), 0, 0)
    # 返回训练损失和训练精度
    return metric[0]
def load_array(features,labels,batch_size, resize=None):  #@save
    dataset = TensorDataset(torch.tensor(features,dtype=torch.float32), torch.tensor(labels,dtype=torch.float32))
    return data.DataLoader(dataset, batch_size, shuffle=True,
                            num_workers=12)

In [7]:
#print(poly_features[:n_train, :4])
#print( labels[:n_train])
train_iter = load_array(poly_features[:n_train, :4], labels[:n_train].reshape(-1,1),10)

x,y=next(iter(train_iter))
print(x,y)

tensor([[ 1.0000e+00, -3.9340e-03,  7.7383e-06, -1.0148e-08],
        [ 1.0000e+00,  2.3875e-02,  2.8501e-04,  2.2682e-06],
        [ 1.0000e+00, -1.0768e+00,  5.7980e-01, -2.0812e-01],
        [ 1.0000e+00, -1.4329e+00,  1.0266e+00, -4.9037e-01],
        [ 1.0000e+00,  1.1835e-01,  7.0030e-03,  2.7626e-04],
        [ 1.0000e+00,  1.0683e+00,  5.7061e-01,  2.0319e-01],
        [ 1.0000e+00,  1.5611e+00,  1.2185e+00,  6.3408e-01],
        [ 1.0000e+00,  3.2656e-01,  5.3320e-02,  5.8041e-03],
        [ 1.0000e+00,  1.0608e+00,  5.6261e-01,  1.9893e-01],
        [ 1.0000e+00, -9.5355e-01,  4.5463e-01, -1.4450e-01]]) tensor([[ 4.9312],
        [ 5.0719],
        [ 0.6250],
        [-3.0737],
        [ 5.1691],
        [ 5.6737],
        [ 6.1107],
        [ 5.1819],
        [ 5.4788],
        [ 1.5250]])


In [8]:
# print(x.shape)
# print(x.shape[-1])
# net=nn.Sequential(nn.Linear(x.shape[-1], 1, bias=False))
# if isinstance(net, torch.nn.Module):
#         net.train()
#     # 训练损失总和、训练准确度总和、样本数
# metric = Accumulator(3)
# #updater=optim.SGD(net.parameters, 0.1, 20)
# #X,y=next(iter(train_iter))
# trainer = torch.optim.SGD(net.parameters(), lr=0.01)
# y_hat = net(x)
# print(y_hat)
# critizen = nn.MSELoss()
# trainer.zero_grad()
# l=critizen(y_hat,y)
# l.backward()
# trainer.step()

In [ ]:
def train(train_features, test_features, train_labels, test_labels,
          num_epochs=60):
    loss = nn.MSELoss(reduction='none')
    input_shape = train_features.shape[-1]
    # 不设置偏置，因为我们已经在多项式中实现了它
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    batch_size = min(10, train_labels.shape[0])
    train_iter = load_array(train_features, train_labels.reshape(-1,1),
                                batch_size)
    test_iter = load_array(test_features, test_labels.reshape(-1,1),
                               batch_size)
    trainer = torch.optim.SGD(net.parameters(), lr=0.01)

    for epoch in range(num_epochs):
        train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 3 == 0:
            print("训练：",evaluate_loss(net, train_iter, loss))
            print("测试：",evaluate_loss(net, test_iter, loss))
    print('weight:', net[0].weight.data.numpy())

# 从多项式特征中选择前4个维度，即1,x,x^2/2!,x^3/3!
train(poly_features[:n_train, :10], poly_features[n_train:, :10],
      labels[:n_train], labels[n_train:],60)

训练： 260.8212537765503
测试： 309.4965295791626
训练： 17.998680233955383
测试： 108.27854335308075
训练： 15.036430448293686
测试： 92.26175925135612
训练： 9.858259081840515
测试： 83.83563834428787
训练： 9.22137701511383
测试： 65.68872049450874


In [ ]:
# 从多项式特征中选择前4个维度，即1,x,x^2/2!,x^3/3!
train(poly_features[:n_train, :4], poly_features[n_train:, :4],
      labels[:n_train], labels[n_train:],20)

TypeError: load_array() missing 1 required positional argument: 'batch_size'